In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 5,
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.10"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 02_portfolio_optimization\n",
        "\n",
        "This notebook demonstrates a simple workflow for running the rolling-horizon\n",
        "Dynamic Mean--Variance optimizer (DynamicMVO) on simulated hybrid price paths.\n",
        "\n",
        "Steps:\n",
        "- load previously generated hybrid simulation outputs (if available), otherwise run a short simulation\n",
        "- construct a small multi-asset price matrix from simulated sample paths\n",
        "- compute log-returns, estimate expected returns and covariance\n",
        "- run DynamicMVO.optimize_over_time\n",
        "- plot portfolio weights over time and cumulative portfolio return curve\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# Imports\n",
        "import os\n",
        "import numpy as np\n",
        "import pandas as pd\n",
        "import matplotlib.pyplot as plt\n",
        "from matplotlib import cm\n",
        "\n",
        "from src.models.heston import HestonModel\n",
        "from src.models.regime_switching import RegimeSwitchingModel\n",
        "from src.models.hybrid_vol_model import HybridVolModel\n",
        "from src.simulation.path_simulator import PathSimulator\n",
        "\n",
        "from src.utils.return_statistics import compute_log_returns, estimate_expected_returns, estimate_covariance_matrix\n",
        "from src.optimization.dynamic_mvo import DynamicMVO\n",
        "\n",
        "%matplotlib inline\n",
        "plt.rcParams['figure.figsize'] = (12, 5)\n",
        "plt.rcParams['font.size'] = 11\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Load or simulate hybrid paths\n",
        "\n",
        "The notebook will try to load arrays saved under `results/` (hybrid S and v paths). If they are not present, it will run a modest simulation here for demonstration purposes."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "results_dir = 'results'\n",
        "os.makedirs(results_dir, exist_ok=True)\n",
        "\n",
        "# Attempt to load precomputed arrays\n",
        "s_path_file = os.path.join(results_dir, 'S_paths.npy')\n",
        "v_path_file = os.path.join(results_dir, 'hybrid_v_paths.npy')\n",
        "regimes_file = os.path.join(results_dir, 'regimes_paths.npy')\n",
        "\n",
        "loaded = False\n",
        "if os.path.exists(s_path_file) and os.path.exists(v_path_file) and os.path.exists(regimes_file):\n",
        "    try:\n",
        "        S_paths = np.load(s_path_file)\n",
        "        hybrid_v_paths = np.load(v_path_file)\n",
        "        regimes_paths = np.load(regimes_file)\n",
        "        print('Loaded simulation outputs from results/')\n",
        "        loaded = True\n",
        "    except Exception as e:\n",
        "        print('Failed to load saved arrays:', e)\n",
        "        loaded = False\n",
        "\n",
        "if not loaded:\n",
        "    # Run a compact simulation for demonstration\n",
        "    n_paths = 200     # number of Monte Carlo sample paths\n",
        "    n_steps = 500\n",
        "    dt = 1.0 / 252.0\n",
        "    seed = 123\n",
        "\n",
        "    # Heston base parameters (example)\n",
        "    heston = HestonModel(kappa=1.5, theta=0.04, sigma=0.3, rho=-0.7, v0=0.04, s0=100.0, r=0.01, dt=dt)\n",
        "    P = np.array([[0.95, 0.04, 0.01], [0.03, 0.94, 0.03], [0.02, 0.08, 0.90]])\n",
        "    vol_factors = np.array([0.6, 1.0, 1.8])\n",
        "    regime_model = RegimeSwitchingModel(P=P, vols=vol_factors)\n",
        "    hybrid = HybridVolModel(heston_model=heston, regime_model=regime_model, vols_are_multiplicative=True)\n",
        "\n",
        "    # Use PathSimulator wrapper to run a hybrid sim\n",
        "    sim = PathSimulator(heston_model=heston, regime_model=regime_model, hybrid_model=hybrid)\n",
        "    hybrid_v_paths, S_paths, regimes_paths = sim.simulate_hybrid_paths(n_paths=n_paths, n_steps=n_steps, seed=seed)\n",
        "\n",
        "    # Save to results for subsequent runs\n",
        "    np.save(s_path_file, S_paths)\n",
        "    np.save(v_path_file, hybrid_v_paths)\n",
        "    np.save(regimes_file, regimes_paths)\n",
        "    print(f\"Ran simulation and saved outputs to {results_dir}/\")\n",
        "\n",
        "# Inspect shapes\n",
        "print('S_paths shape:', S_paths.shape)          # (n_paths, n_steps+1)\n",
        "print('hybrid_v_paths shape:', hybrid_v_paths.shape)\n",
        "print('regimes_paths shape:', regimes_paths.shape)\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Build a multi-asset price matrix\n",
        "\n",
        "We construct a modest multi-asset price matrix for the optimizer by treating several Monte Carlo sample paths as distinct assets (this is a convenient way to obtain multiple correlated/heteroskedastic price series for demonstration)."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# Select the first `n_assets` simulated sample paths as synthetic assets\n",
        "n_assets = 5\n",
        "if S_paths.shape[0] < n_assets:\n",
        "    raise RuntimeError('Not enough simulated paths to form multi-asset matrix')\n",
        "\n",
        "price_matrix = S_paths[:n_assets, :]  # shape (n_assets, T+1)\n",
        "T = price_matrix.shape[1] - 1\n",
        "print('Price matrix shape (n_assets, T+1):', price_matrix.shape)\n",
        "\n",
        "# Visual quick-check: plot the selected asset price paths\n",
        "t = np.arange(0, T + 1) * (1.0 / 252.0)\n",
        "fig, ax = plt.subplots(figsize=(10, 4))\n",
        "for i in range(n_assets):\n",
        "    ax.plot(t, price_matrix[i], label=f'asset_{i}', alpha=0.9)\n",
        "ax.set_title('Synthetic multi-asset price series (first sample paths)')\n",
        "ax.set_xlabel('Time (years)')\n",
        "ax.set_ylabel('Price')\n",
        "ax.legend(loc='upper left')\n",
        "ax.grid(alpha=0.2)\n",
        "plt.show()\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Compute returns, expected returns, and covariance\n",
        "\n",
        "We compute log-returns from the price matrix and estimate expected returns and the covariance matrix used by the MVO optimizer."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# Compute log returns: returns shape (n_assets, T)\n",
        "returns = compute_log_returns(price_matrix)\n",
        "print('returns shape:', returns.shape)\n",
        "\n",
        "# Estimate expected returns (mean across time for each asset)\n",
        "mu_hat = estimate_expected_returns(returns, axis=1)  # shape (n_assets,)\n",
        "print('Estimated expected returns (mu_hat):', np.round(mu_hat, 6))\n",
        "\n",
        "# Estimate covariance matrix across assets (observations are columns -> rowvar=True)\n",
        "Sigma_hat = estimate_covariance_matrix(returns, rowvar=True, ddof=1)\n",
        "print('Covariance matrix shape:', Sigma_hat.shape)\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Run Dynamic MVO (rolling-horizon)\n",
        "\n",
        "We instantiate DynamicMVO and run the rolling optimization over the price matrix.\n",
        "The optimizer returns a time series of weights (n_assets x T) and the portfolio wealth curve (length T+1)."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# Configure optimizer\n",
        "lambda_risk_aversion = 10.0\n",
        "window = 20  # historical window (in returns) used for rolling estimates\n",
        "\n",
        "optimizer = DynamicMVO(risk_aversion=lambda_risk_aversion, regularization=1e-6, bounds=(0.0, 1.0))\n",
        "\n",
        "# Run rolling optimization\n",
        "weights_ts, portfolio_values = optimizer.optimize_over_time(price_matrix, window=window)\n",
        "print('weights_ts shape:', weights_ts.shape)  # (n_assets, T)\n",
        "print('portfolio_values shape:', portfolio_values.shape)  # (T+1,)\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Plot portfolio weights over time\n",
        "\n",
        "We show the time evolution of the optimizer's weights using a stacked area chart."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "time = np.arange(0, returns.shape[1]) * (1.0 / 252.0)\n",
        "labels = [f'asset_{i}' for i in range(n_assets)]\n",
        "colors = cm.get_cmap('tab10')(np.arange(n_assets))\n",
        "\n",
        "fig, ax = plt.subplots(figsize=(12, 4))\n",
        "ax.stackplot(time, weights_ts, labels=labels, colors=colors, alpha=0.85)\n",
        "ax.set_title('Dynamic MVO weights over time (rolling window=%d)' % window)\n",
        "ax.set_xlabel('Time (years)')\n",
        "ax.set_ylabel('Portfolio weight')\n",
        "ax.legend(loc='upper right')\n",
        "ax.grid(alpha=0.15)\n",
        "plt.show()\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Plot cumulative portfolio return curve\n",
        "\n",
        "Portfolio values were initialized at 1.0. We plot the wealth curve over time to visualize cumulative performance."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "t_full = np.arange(0, portfolio_values.shape[0]) * (1.0 / 252.0)\n",
        "fig, ax = plt.subplots(figsize=(10, 4))\n",
        "ax.plot(t_full, portfolio_values, lw=2, color='C0')\n",
        "ax.set_title('Portfolio wealth curve (Dynamic MVO)')\n",
        "ax.set_xlabel('Time (years)')\n",
        "ax.set_ylabel('Wealth')\n",
        "ax.grid(alpha=0.2)\n",
        "plt.show()\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Notes and next steps\n",
        "- This notebook uses a convenient approach (treating Monte Carlo sample paths as synthetic assets) for demonstration. For realistic multi-asset experiments, replace the price matrix with actual asset price histories or simulate correlated multi-asset processes.\n",
        "- Consider adding transaction cost modeling and turnover-aware objective adjustments for more realistic backtests.\n",
        "- Save the produced plots and numeric outputs under `results/` for reproducibility of figures in analysis notebooks."
      ]
    }
  ]
}